#### Задача 1
Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [67]:
# Функция парсинга рейтингов молочных продуктов с сайта Росконтроль,
# которая возвращает список словарей с параметрами продукта

def food_rater(product:str): 
    import requests
    from bs4 import BeautifulSoup as bs
    import pandas as pd
    
    
    url = f'https://roscontrol.com/category/produkti/molochnie_produkti/{product}'
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    soup = bs(response.content, 'html.parser')
    item = soup.find_all('h2', class_='woocommerce-loop-product__title')
    items = []
    for row in item:
        items.append(row.text)
    value = soup.find_all('p', attrs={'class':['rating_one_criterion']})
    values = []
    for row in value:
        values.append(row.find(string=True))
    values = list(map(float, values))
    natural = soup.find_all('p', attrs={'class':['rating_two_criterion']})
    naturals = []
    for row in natural:
        naturals.append(row.find(string=True))
    naturals = list(map(float, naturals))
    safety = soup.find_all('p', attrs={'class':['rating_three_criterion']})
    safeties = []
    for row in safety:
        safeties.append(row.find(string=True))
    safeties = list(map(float, safeties))
    quality = soup.find_all('p', attrs={'class':['rating_four_criterion']})
    qualities = []
    for row in quality:
        qualities.append(row.find(string=True))
    qualities = list(map(float, qualities))
    link = soup.find_all('a', attrs={'class':['woocommerce-LoopProduct-link woocommerce-loop-product__link']})
    links = []
    for row in link:
        links.append(row.get('href'))
    products = []
    for i in range(len(items)):
        products.append({'Название': items[i], 'Пищевая ценность': values[i], 'Натуральность': naturals[i],
                     'Безопасность': safeties[i], 'Качество': qualities[i], 'Ссылка':links[i]})
    return products

In [68]:
from pprint import pprint


products = food_rater('yogurti') # запишем в переменную результат выполнения парсинга 

In [72]:
# Функция добавления в базу новых (ранее не добавленных) продуктов

def add_new_prods(list:products):
    from pymongo import MongoClient
    
    
    client = MongoClient()
    db = client['products']
    collection = db['milk']
    for i in products:
        if collection.count_documents({'Название': i['Название']})==0:
            collection.insert_one(i)
    print('Добавление в базу новых продуктов выполнено')

In [77]:
add_new_prods(products)

Добавление в базу новых продуктов выполнено


In [80]:
for i in collection.find():
    pprint(i)

{'_id': ObjectId('63f7e6b37f01de96976bc0a1'),
 'Безопасность': 9.5,
 'Качество': 8.8,
 'Название': 'Йогурт «Агуша» с персиком',
 'Натуральность': 5.7,
 'Пищевая ценность': 9.3,
 'Ссылка': 'https://roscontrol.com/product/yogurt-agusha-s-persikom/'}
{'_id': ObjectId('63f7e6b37f01de96976bc0a2'),
 'Безопасность': 8.0,
 'Качество': 9.2,
 'Название': '«Активиа» термостатная 3,5%',
 'Натуральность': 6.5,
 'Пищевая ценность': 7.7,
 'Ссылка': 'https://roscontrol.com/product/bioioghurt-aktivia-tiermostatnaia-3-5/'}
{'_id': ObjectId('63f7e6b37f01de96976bc0a3'),
 'Безопасность': 8.7,
 'Качество': 6.0,
 'Название': 'Йогурт греческий Teos',
 'Натуральность': 8.5,
 'Пищевая ценность': 7.5,
 'Ссылка': 'https://roscontrol.com/product/yogurt-grecheskiy-teos/'}
{'_id': ObjectId('63f7e6b37f01de96976bc0a4'),
 'Безопасность': 8.7,
 'Качество': 6.0,
 'Название': '«Савушкин» «Греческий Teos» с наполнителем черника',
 'Натуральность': 8.5,
 'Пищевая ценность': 7.5,
 'Ссылка': 'https://roscontrol.com/product/sa

#### Задача 2
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [90]:
# Функция отбора и вывода на экран продуктов с общим рейтингом не ниже введенного
# или качеством не ниже ввденного.

def weed_out(n):
    from pymongo import MongoClient
    from pprint import pprint
    
    
    client = MongoClient()
    db = client['products']
    collection = db['milk']
    for i in collection.find():
        if (i['Качество'] >= n or (i['Качество']+i['Безопасность']+i['Натуральность']+i['Пищевая ценность'])/4 >= n):
            pprint(i)

In [91]:
weed_out(8)

{'_id': ObjectId('63f7e6b37f01de96976bc0a1'),
 'Безопасность': 9.5,
 'Качество': 8.8,
 'Название': 'Йогурт «Агуша» с персиком',
 'Натуральность': 5.7,
 'Пищевая ценность': 9.3,
 'Ссылка': 'https://roscontrol.com/product/yogurt-agusha-s-persikom/'}
{'_id': ObjectId('63f7e6b37f01de96976bc0a2'),
 'Безопасность': 8.0,
 'Качество': 9.2,
 'Название': '«Активиа» термостатная 3,5%',
 'Натуральность': 6.5,
 'Пищевая ценность': 7.7,
 'Ссылка': 'https://roscontrol.com/product/bioioghurt-aktivia-tiermostatnaia-3-5/'}
